# Analyse d'image de marque sur les réseaux sociaux

### Importation des Bibliothèques et du Fichier 'utilities'

Dans cette section, nous importons les bibliothèques Python essentielles pour notre analyse de traitement naturel du langage (NLP) et nos tâches de *machine learning*. Ces bibliothèques comprennent des outils pour le prétraitement des données, la modélisation, l'évaluation des modèles, et bien d'autres. Nous avons également créé un fichier nommé 'utilities.py' qui contient des méthodes personnalisées que nous utiliserons tout au long du projet.

In [ ]:
import utilities
import pandas            as pd
import numpy             as np
import seaborn           as sns
import matplotlib.pyplot as plt
import re
import sklearn
import random
import textdistance
import umap
import pickle

from sklearn.tree              import DecisionTreeClassifier
from sklearn.model_selection   import train_test_split
from sklearn                   import metrics
from sklearn.model_selection   import GridSearchCV
from transformers              import BertTokenizer, BertModel
from sentence_transformers     import SentenceTransformer
from sklearn.metrics           import roc_curve
from umap                      import UMAP
from typing                    import Optional
from sklearn.decomposition     import PCA
from sklearn.linear_model      import LogisticRegression
from sklearn.metrics           import confusion_matrix

### Import des données

In [ ]:
corpus = pd.read_csv("C:/Users/a.tekiouk/Sujet_2/Sujet_2/DATA/chanel10k.csv", sep=";", parse_dates=["publication_time"])

# on récupére les posts en anglais
mask = corpus["language"] == 'en'
publication_time_chanel = corpus['publication_time'].tolist()

print(f"# documents in corpus: {len(chanel = corpus['text'].tolist())}")

### Importation du Corpus de Publications Labellisées

Dans cette section, nous procédons à l'importation du corpus de 500 publications labellisées comme "junk" (indésirables) ou "non-junk" (acceptables). Ces données sont essentielles pour notre projet, car elles serviront de base pour l'entraînement et l'évaluation de nos modèles de détection de publications indésirables.  
  
Avant d'importer les données, nous avons effectué une étape cruciale de prétraitement en utilisant une méthode personnalisée que nous avons définie dans le fichier 'utilities.py'. Cette méthode, appelée `delete_duplicates`, a permis de supprimer efficacement les doublons et les publications quasi-doublons de notre corpus. Cette opération est importante pour éviter tout biais dans notre analyse, car des publications similaires pourraient fausser les résultats de nos modèles.  
  
Le corpus de données que nous importons ici est prêt à être utilisé pour la caractérisation textuelle, la modélisation, et l'évaluation de nos modèles de détection de publications indésirables. Nous pouvons maintenant procéder à l'exploration de ces données et à la mise en place des étapes suivantes de notre projet.


In [ ]:
chanel_junk_valid_new = pd.read_excel('C:/Users/a.tekiouk/Sujet_2/DATA/chanel_junk_valid_2.xlsx')
chanel_junk_valid_new = chanel_junk_valid_new[['text','is_junk']].dropna()

## Caractérisation manuelle du texte

### Création de la Pipeline spaCy 'nlp'

Dans cette section, nous mettons en place une pipeline spaCy 'nlp' personnalisée qui nous permettra de traiter et d'analyser nos publications textuelles. SpaCy est une bibliothèque NLP (Traitement du Langage Naturel) puissante et polyvalente.

Tout d'abord, nous importons spaCy et initialisons la pipeline 'nlp'. Cette pipeline comprendra plusieurs étapes de prétraitement et d'analyse du texte, ce qui facilitera notre travail d'extraction d'informations et de caractérisation des publications.

Une des étapes importantes que nous ajoutons à cette pipeline est la reconnaissance des tokens 'hashtags' et 'emojis'. Les hashtags (#) sont couramment utilisés sur les réseaux sociaux pour identifier des sujets spécifiques, tandis que les emojis ajoutent une dimension émotionnelle à un texte. En identifiant ces éléments, nous pouvons mieux comprendre le contenu des publications, ce qui est essentiel pour notre analyse.

Nous personnalisons notre pipeline spaCy pour répondre aux besoins spécifiques de notre projet, ce qui nous permettra d'extraire efficacement les informations pertinentes des publications et de les utiliser dans nos modèles de détection de publications indésirables.

In [ ]:
nlp = spacy.load("en_core_web_sm")

@Language.component("hashtag")
def hashtag_pipe(
    doc: spacy.tokens.Doc
) -> spacy.tokens.Doc:
    """
    A spaCy pipeline component that merges tokens of the form #word into a single token.

    Parameters
    ----------
    doc : spacy.tokens.Doc
        The input spaCy Doc object to process.

    Returns
    -------
    spacy.tokens.Doc
        The processed spaCy Doc object with hashtags merged into a single token.
    """
    len_doc = -1
    for token in doc:
        len_doc=len_doc+1
    merged_hashtag = False
    while True:
        for token in doc:
            if token.text == '#':
                if(token.head is not None and token.i!=len_doc):
                    start_index = token.i
                    end_index = start_index + 1
                    with doc.retokenize() as retokenizer:
                        retokenizer.merge(doc[start_index:end_index+1])
                        merged_hashtag = True
                        break
        if not merged_hashtag:
            break
        merged_hashtag = False
    return doc

nlp.add_pipe("emoji", first=True)
nlp.add_pipe("hashtag", first=True)
Token.set_extension("is_hashtag", getter=lambda token: token.text[0] in ("#"), force=True)

### Création de Variables Caractéristiques

Dans cette section, nous créons plusieurs variables caractéristiques à partir des publications de notre corpus. Ces variables nous permettront de mieux caractériser chaque publication et de les utiliser dans nos modèles de détection de publications indésirables.

- **has_URL** : Cette variable binaire indique si une publication contient ou non une URL (Uniform Resource Locator). La présence d'URLs peut être un indicateur de publications promotionnelles ou de spam.  

- **has_phone_number** : Cette variable binaire indique si une publication contient ou non un numéro de téléphone. La présence de numéros de téléphone peut être associée à des publications de type publicité ou spam.  

- **has_currency_symbol** : Cette variable binaire indique si une publication contient ou non un symbole monétaire (comme le dollar ou l'euro). La présence de symboles monétaires peut être liée à des offres commerciales ou publicitaires.  

- **word_ratio** : Cette variable quantifie la répartition des mots par rapport à l'ensemble des éléments textuels (mots, ponctuations, emojis, hashtags, etc.) dans une publication. Elle peut nous donner des indications sur la densité du texte et sa structure. Par exemple, une faible densité de mots par rapport aux autres éléments peut suggérer une publication dominée par des éléments non textuels tels que des emojis et des hashtags.



In [ ]:
chanel_junk_valid_new = pd.read_excel('C:/Users/a.tekiouk/Sujet_2/DATA/chanel_junk_valid_2.xlsx')
chanel_junk_valid_new = chanel_junk_valid_new[['text','is_junk']].dropna()
chanel_junk_valid_new['has_URL'] = utilities.get_presence_of_URL(corpus= chanel_junk_valid_new['text'], nlp= nlp  )
chanel_junk_valid_new['has_phone_number'] = utilities.get_presence_of_phone_numbers(corpus= chanel_junk_valid_new['text'], nlp= nlp  )
chanel_junk_valid_new['has_currency_symbol'] = utilities.get_presence_of_currency_symbol(corpus= chanel_junk_valid_new['text'], nlp= nlp  )
chanel_junk_valid_new['word_ratio'] = utilities.get_word_ratio(corpus= chanel_junk_valid_new['text'], nlp= nlp  )
chanel_junk_valid_new = chanel_junk_valid_new.join(utilities.create_dummies(corpus= chanel_junk_valid_new['text'],y = chanel_junk_valid_new['is_junk'], element= 'emoji',nlp = nlp, top= 3))
chanel_all_features = chanel_junk_valid_new.join(utilities.create_dummies(corpus= chanel_junk_valid_new['text'],y = chanel_junk_valid_new['is_junk'], element= 'hashtag', nlp = nlp, top= 3))
X_train, X_test, y_train, y_test = train_test_split(chanel_all_features.dropna().drop(['text','is_junk'],axis=1), chanel_all_features.dropna()['is_junk'], test_size=0.2, random_state=42)

### Optimisation des Hyperparamètres

Dans cette section, nous détaillons le processus d'optimisation des hyperparamètres pour nos modèles de détection de publications indésirables.

1. **Optimisation en Fonction de l'AUC**

    - Nous commençons par rechercher les meilleures combinaisons d'hyperparamètres pour nos modèles en fonction de l'AUC (Area Under the Curve). L'AUC mesure la capacité d'un modèle à bien classer les observations positives et négatives.
    
    - Nous utilisons des méthodes d'optimisation telles que la recherche en grille (*GridSearch*) pour explorer différentes valeurs d'hyperparamètres.
    
    - L'objectif principal est de maximiser l'AUC, ce qui garantit une capacité globale de classification précise de nos modèles.

2. **Optimisation en Fonction de la Précision**

    - Après avoir obtenu les meilleures configurations d'hyperparamètres en fonction de l'AUC, nous passons à l'optimisation en fonction de la précision.
    
    - La précision est essentielle pour minimiser les faux positifs, c'est-à-dire les publications incorrectement identifiées comme indésirables.
    
    - Nous ajustons les hyperparamètres autour des valeurs optimales précédemment trouvées tout en mettant l'accent sur l'amélioration de la précision.
    

In [ ]:
param_grid = {  'criterion': ['entropy'],
                'ccp_alpha' : np.linspace(0,0.5,50),
                'min_samples_leaf' : np.linspace(2,15,14,dtype= int),
                'min_samples_split' : np.linspace(5,25,21,dtype= int)
                }  
   
grid = GridSearchCV(DecisionTreeClassifier(random_state = 42), param_grid, scoring = 'roc_auc' ,refit = True, verbose = 3,n_jobs=-1)
grid.fit(X_train, y_train)
print(grid.best_params_)

In [ ]:
param_grid_ccp = {  'criterion': ['entropy'],
                    'ccp_alpha' : np.linspace(0.0,0.01,40),
                    'min_samples_leaf' : np.linspace(3,5,3,dtype= int),
                    'min_samples_split' : np.linspace(19,21,3,dtype= int)
                    }  

   
grid_ccp = GridSearchCV(DecisionTreeClassifier(random_state=42),param_grid_ccp, scoring = 'precision' ,refit = True, verbose = 3,n_jobs=-1)
grid_ccp.fit(X_train, y_train)
print(grid_ccp.best_params_)

In [ ]:
best_params = grid_ccp.best_params_
best_params['random_state']=42
tree_f = DecisionTreeClassifier(**best_params)
tree_f.fit(X_train,y_train)

*Serialization* : 

In [ ]:
with open('C:/Users/a.tekiouk/Sujet_2/Models/model_tree_auc_spacy.pkl', 'wb') as f:  # open a text file
    pickle.dump(tree_f, f) # serialize the model

In [ ]:
y_pred = tree_f.predict(X_test)
utilities.evaluate(y_test=y_test,
                   y_pred=y_pred,
                   y_score=tree_f.predict_proba(X_test)[:, 1])

In [ ]:
y_score = model_tree_auc_spacy.predict_proba(X_test)[:,1]
utilities.plot_roc(y_test,y_score,"lightblue")

## Caractérisation automatique du texte

In [ ]:
ENCODER_ID = "all-MiniLM-L6-v2"
encoder = SentenceTransformer(ENCODER_ID)

X = encoder.encode(chanel_junk_valid_new['text'])
y = chanel_junk_valid_new['is_junk']

In [ ]:
best_model_auc,best_auc,opt_dim,reducer = utilities.auto_model_selection(
    X=X,
    y=y,
    dimension_range=range(10,160,10))
print(f"best model : {best_model_auc}, hyperparameters = {best_model_auc.get_params()}\nAUC : {best_auc}\nOptimal number of dimensions : {opt_dim}\nBest reducing method : {reducer}")

*Serialization* : 

In [ ]:
with open('C:/Users/a.tekiouk/Sujet_2/Models/model_auc.pkl', 'wb') as f:  # open a text file
    pickle.dump(best_model_auc, f) # serialize the model
with open('C:/Users/a.tekiouk/Sujet_2/Models/opt_reducer.pkl', 'wb') as f:  # open a text file
    pickle.dump(reducer, f) # serialize the reducer